In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import os
from jinja2 import Environment, FileSystemLoader
from pathlib import Path
from tqdm.notebook import tqdm
import time
import sys
import json
from sklearn.metrics import accuracy_score

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.utils import convert_types
from utils.vars import DATA_DIR, EXCEPT_FILES, QUESTION_PATH

load_dotenv()

True

In [2]:
# read questions
df_questions = pd.read_csv(QUESTION_PATH)

# get the client object
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-10-01-preview",  # different from assistant
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

## Using the same prompt in the function_instruction.jinja2 file

- OpenAI models cannot run the code but can generate the code correctly
- The #tokens are a lot 
- The execution time is much longer: 20s for one question for melborune_temp
- Cannot run ny_taxi.csv due to token limit
- data is not corrected in the code
- The code overall looks right but does not round to 2 as stated in the prompt

- [TODO] Is there any other way to pass the dataframe

In [ ]:
# read the prompt
instruction = (
    Environment(loader=FileSystemLoader("../function_calling/prompt/"))
    .get_template("code_interpreter_instruction.jinja2")
    .render()
)

# token
df_token = []

for file_path in Path(DATA_DIR).glob("*.csv"):
    if file_path.name in EXCEPT_FILES + ["nyc_taxi.csv"]:
        continue
    print(f"file: {file_path.name}")

    # read the data
    df = pd.read_csv(file_path)

    # call openai
    for question in tqdm(df_questions["question"].tolist()[:5]):
        start_time = time.time()
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": instruction},
                {
                    "role": "user",
                    "content": f"Here is the dataset in the markdown format. {df.to_markdown()}",
                },
                {"role": "user", "content": question},
            ],
            temperature=0,
            top_p=1,
            seed=42,
        )
        df_token.append(
            {
                **response.usage.to_dict(),
                "question": question,
                "execution_time_s": round(time.time() - start_time, 2),
                "file": file_path.name,
            }
        )

        print("Question:", question)
        print(response.choices[0].message.content)
        print("*" * 50)


df_token = pd.DataFrame(df_token)

file: air_passengers.csv


  0%|          | 0/5 [00:00<?, ?it/s]

Question: What is the min of the target variable?
```python
import pandas as pd

# Creating the dataset
data = {
    'ds': ["1949-01-01", "1949-02-01", "1949-03-01", "1949-04-01", "1949-05-01", "1949-06-01", "1949-07-01", "1949-08-01", "1949-09-01", "1949-10-01", "1949-11-01", "1949-12-01", 
           "1950-01-01", "1950-02-01", "1950-03-01", "1950-04-01", "1950-05-01", "1950-06-01", "1950-07-01", "1950-08-01", "1950-09-01", "1950-10-01", "1950-11-01", "1950-12-01", 
           "1951-01-01", "1951-02-01", "1951-03-01", "1951-04-01", "1951-05-01", "1951-06-01", "1951-07-01", "1951-08-01", "1951-09-01", "1951-10-01", "1951-11-01", "1951-12-01", 
           "1952-01-01", "1952-02-01", "1952-03-01", "1952-04-01", "1952-05-01", "1952-06-01", "1952-07-01", "1952-08-01", "1952-09-01", "1952-10-01", "1952-11-01", "1952-12-01", 
           "1953-01-01", "1953-02-01", "1953-03-01", "1953-04-01", "1953-05-01", "1953-06-01", "1953-07-01", "1953-08-01", "1953-09-01", "1953-10-01", "1953-11-01", "1

  0%|          | 0/5 [00:00<?, ?it/s]

Question: What is the min of the target variable?
```python
import pandas as pd

# Creating the dataset
data = {
    "Date": ["1981-01-01", "1981-01-02", "1981-01-03", "1981-01-04", "1981-01-05", "1981-01-06", "1981-01-07", "1981-01-08", "1981-01-09", "1981-01-10", 
             "1981-01-11", "1981-01-12", "1981-01-13", "1981-01-14", "1981-01-15", "1981-01-16", "1981-01-17", "1981-01-18", "1981-01-19", "1981-01-20", 
             "1981-01-21", "1981-01-22", "1981-01-23", "1981-01-24", "1981-01-25", "1981-01-26", "1981-01-27", "1981-01-28", "1981-01-29", "1981-01-30", 
             "1981-01-31", "1981-02-01", "1981-02-02", "1981-02-03", "1981-02-04", "1981-02-05", "1981-02-06", "1981-02-07", "1981-02-08", "1981-02-09", 
             "1981-02-10", "1981-02-11", "1981-02-12", "1981-02-13", "1981-02-14", "1981-02-15", "1981-02-16", "1981-02-17", "1981-02-18", "1981-02-19", 
             "1981-02-20", "1981-02-21", "1981-02-22", "1981-02-23", "1981-02-24", "1981-02-25", "1981-02-26", "1981-

In [4]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_token.groupby(["file"]).describe())

completion_tokens                                     \
                               count    mean        std     min     25%   
file                                                                      
air_passengers.csv               5.0  1689.0  19.429359  1657.0  1685.0   
melbourne_temp.csv               5.0  4096.0   0.000000  4096.0  4096.0   

                                           prompt_tokens                     \
                       50%     75%     max         count     mean       std   
file                                                                          
air_passengers.csv  1696.0  1703.0  1704.0           5.0   2451.2  0.447214   
melbourne_temp.csv  4096.0  4096.0  4096.0           5.0  69088.2  0.447214   

                                                                total_tokens  \
                        min      25%      50%      75%      max        count   
file                                                                           
air_passengers.csv   2451.0   2451.0   2451.0   2451.0   2452.0          5.0   
melbourne_temp.csv  69088.0  69088.0  69088.0  69088.0  69089.0          5.0   

                                                                            \
                       mean        std      min      25%      50%      75%   
file                                                                         
air_passengers.csv   4140.2  19.382982   4108.0   4137.0   4147.0   4154.0   
melbourne_temp.csv  73184.2   0.447214  73184.0  73184.0  73184.0  73184.0   

                            execution_time_s                              \
                        max            count     mean        std     min   
file                                                                       
air_passengers.csv   4155.0              5.0   19.638   3.430936   15.44   
melbourne_temp.csv  73185.0              5.0  236.036  28.925987  198.74   

                                                    
                       25%     50%     75%     max  
file                                                
air_passengers.csv   18.02   19.12   21.00   24.61  
melbourne_temp.csv  215.80  239.44  257.19  269.01

## Using different prompts

In [5]:
prompt_path = "prompt/prompts.jinja2"

In [6]:
# read the prompt
instruction = (
    Environment(loader=FileSystemLoader(".")).get_template(prompt_path).render()
)

# token
df_result = []

for file_path in Path(DATA_DIR).glob("*.csv"):
    if file_path.name in EXCEPT_FILES + ["nyc_taxi.csv"]:
        continue
    print(f"file: {file_path.name}")
    # read the data
    df = pd.read_csv(file_path)

    # call openai
    for _, row in tqdm(df_questions.iterrows(), total=len(df_questions)):
        question = row["question"]
        answer_true = row[Path(file_path).name]

        start_time = time.time()

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": instruction},
                {
                    "role": "user",
                    "content": f"Here is the dataset in the markdown format. {df.to_markdown()}",
                },
                {"role": "user", "content": question},
            ],
            temperature=0,
            top_p=1,
            seed=42,
        )

        # format the output
        try:
            answer_pred = json.loads(response.choices[0].message.content)["output"]
        except json.decoder.JSONDecodeError:
            print(
                f"Original output: {response.choices[0].message.content}; JSONDecodeError: {answer_pred}"
            )

        df_result.append(
            {
                **response.usage.to_dict(),
                "question": question,
                "execution_time_s": round(time.time() - start_time, 2),
                "file": file_path.name,
                "answer_pred": convert_types(answer_pred),
                "answer_true": convert_types(answer_true),
            }
        )

df_result = pd.DataFrame(df_result)

for file in df_result["file"].unique():
    df_tmp = df_result[df_result["file"] == file]
    print(
        f"File: {file}; Accuracy: {accuracy_score(df_tmp['answer_true'].astype(str).tolist(), df_tmp['answer_pred'].astype(str).tolist())}"
    )
    for _, row in df_tmp[df_tmp["answer_true"] != df_tmp["answer_pred"]].iterrows():
        print(f"question: {row['question']}")
        print(f"answer_pred: {row['answer_pred']}; answer_true: {row['answer_true']}")
    del df_tmp
    print("*" * 50)

file: air_passengers.csv


  0%|          | 0/16 [00:00<?, ?it/s]

file: melbourne_temp.csv


  0%|          | 0/16 [00:00<?, ?it/s]

Original output: ```json
{"output": "0"}
```; JSONDecodeError: 8
File: air_passengers.csv; Accuracy: 0.875
question: What is the Q3 of the target variable?
answer_pred: 355.5; answer_true: 360.5
question: What is the frequency of the given time series data?
answer_pred: monthly; answer_true: MS
**************************************************
File: melbourne_temp.csv; Accuracy: 0.5625
question: What is the medium of the target variable?
answer_pred: 12.3; answer_true: 11.0
question: What is the standard deviation of the target variable?
answer_pred: 4.34; answer_true: 4.07
question: What is the Q1 of the target variable?
answer_pred: 9.6; answer_true: 8.3
question: What is the Q3 of the target variable?
answer_pred: 16.4; answer_true: 14.0
question: How many outliers in the the target variable?
answer_pred: 8.0; answer_true: 13.0
question: How many missing dates in the time column?
answer_pred: 8.0; answer_true: 0.0
question: What is the frequency of the given time series data?
answe

In [7]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_result.groupby(["file"]).describe())

completion_tokens                                         \
                               count    mean       std  min   25%  50%  75%   
file                                                                          
air_passengers.csv              16.0  7.2500  1.732051  6.0  6.00  6.0  8.0   
melbourne_temp.csv              16.0  7.9375  1.611159  6.0  6.75  8.0  8.0   

                         prompt_tokens                                         \
                     max         count      mean       std      min       25%   
file                                                                            
air_passengers.csv  11.0          16.0   2437.25  1.732051   2434.0   2436.75   
melbourne_temp.csv  11.0          16.0  69074.25  1.732051  69071.0  69073.75   

                                              total_tokens              \
                        50%      75%      max        count        mean   
file                                                                     
air_passengers.csv   2437.0   2438.0   2440.0         16.0   2444.5000   
melbourne_temp.csv  69074.0  69075.0  69077.0         16.0  69082.1875   

                                                                            \
                         std      min       25%      50%      75%      max   
file                                                                         
air_passengers.csv  3.183290   2440.0   2442.75   2444.5   2446.0   2451.0   
melbourne_temp.csv  2.903303  69077.0  69080.75  69082.0  69083.0  69088.0   

                   execution_time_s                                          \
                              count      mean       std   min     25%   50%   
file                                                                          
air_passengers.csv             16.0  0.343125  0.120151 -0.07  0.3275  0.37   
melbourne_temp.csv             16.0  7.735625  3.754386  1.22  6.3400  7.10   

                                   
                       75%    max  
file                               
air_passengers.csv  0.3925   0.49  
melbourne_temp.csv  8.6150  15.81